In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal
import seaborn as sns

In [ ]:
a = np.array([-.3, .5])

beta = (1/.2)**2  # Precision parameter (noise variance)
alpha = 2.  # precision of prior 

In [ ]:
def get_y(x, a):
    return a[0] + a[1] * x

In [ ]:
def generate_synthetic_data(a, beta, n=1):
    x = np.random.uniform(-1, 1, n)
    t = get_y(x, a) + np.random.normal(0, 1/ np.sqrt(beta))
    return x, t

In [ ]:
generate_synthetic_data(a,1)

In [ ]:
# plot the prior 
w0, w1 = np.mgrid[-1:1:.01, -1:1:.01]
pos = np.empty(w0.shape + (2,))
pos[:, :, 0] = w0; pos[:, :, 1] = w1
rv = multivariate_normal([0, 0], (1/ alpha) * np.eye(2))

plt.imshow(rv.pdf(pos), extent=[-1, 1, -1 ,1], cmap='inferno')

plt.title('Prior distribution')

plt.xlabel('w_0')
plt.ylabel('w_1')

In [ ]:
samples = rv.rvs(6)

In [ ]:
plt.figure(figsize=[4,4])

x = np.linspace(-1, 1, 50)

for s in samples:
    plt.plot(x, s[0] + x * s[1])

plt.xlim([-1,1])
plt.ylim([-1,1])

In [ ]:
# Now we observe a single data point 
x, t = generate_synthetic_data(a, beta, 1)

In [ ]:
# We can therefore evaluate the likelihood function p(t| x, w)

In [ ]:
def likelihood_func(t, x, w, beta):
    rv = multivariate_normal(get_y(x, w), 1/beta)
    return rv.pdf(t)

In [ ]:
likelihood_mat = np.empty(w1.shape)
for i, wx in enumerate(w1[0]):
    for j, wy in enumerate(w1[0]):
        likelihood = likelihood_func(t[0], x[0], np.array([wx, wy]), beta)
        likelihood_mat[i, j] = likelihood

In [ ]:
likelihood_mat.shape

In [ ]:
plt.imshow(likelihood_mat, extent=[-1, 1, -1, 1])

plt.scatter(a[0], a[1], marker='+', c='white', s=100)
plt.title('Likelihood')


plt.xlabel('w_0')
plt.ylabel('w_1')

In [ ]:
# Now, we want to compute a posterior distribution by multiplying the likelihood function by the prior, 
# and normalising. 

In [ ]:
# plot the posterior 

# new parameters (can just update parameters because we use a conjugate prior)
S_N_inv = alpha * np.eye(2) + beta * np.outer(np.array([1, x]), np.array([1, x]))
S_N = np.linalg.inv(S_N_inv)
m = beta * np.matmul(S_N, np.array([1,x])) * t

In [ ]:
w0, w1 = np.mgrid[-1:1:.01, -1:1:.01]
pos = np.empty(w0.shape + (2,))
pos[:, :, 0] = w0; pos[:, :, 1] = w1
rv = multivariate_normal(m, S_N)

plt.imshow(rv.pdf(pos), extent=[-1, 1, -1 ,1], cmap='inferno')
plt.scatter(a[0], a[1], marker='+', c='white', s=100)


plt.title('Posterior distribution')

plt.xlabel('w_0')
plt.ylabel('w_1')

In [ ]:
posterior_samples = rv.rvs(6)

In [ ]:
plt.figure(figsize=[4,4])

xs = np.linspace(-1, 1, 50)

for s in posterior_samples:
    plt.plot(xs, s[0] + xs * s[1])

plt.scatter(x, t, marker='o')    

plt.xlim([-1,1])
plt.ylim([-1,1])

In [ ]:
# Now we observe a second data point

In [ ]:
# Now we observe a single data point 
x2, t2 = generate_synthetic_data(a, beta, 1)

# the posterior of the past trial is now the prior:
S_N_inv_prior = S_N_inv
S_N_prior = S_N
m_prior = m

In [ ]:
likelihood_mat = np.empty(w1.shape)
for i, wx in enumerate(w1[0]):
    for j, wy in enumerate(w1[0]):
        likelihood = likelihood_func(t2[0], x2[0], np.array([wx, wy]), beta)
        likelihood_mat[i, j] = likelihood

In [ ]:
plt.imshow(likelihood_mat, extent=[-1, 1, -1, 1])

plt.scatter(a[0], a[1], marker='+', c='white', s=100)
plt.title('Likelihood')


plt.xlabel('w_0')
plt.ylabel('w_1')

In [ ]:
# new parameters (can just update parameters because we use a conjugate prior)
S_N_inv = S_N_inv_prior + beta * np.outer(np.array([1, x2]), np.array([1, x2]))
S_N = np.linalg.inv(S_N_inv)
m = np.matmul(S_N, np.matmul(S_N_inv_prior, m_prior) + beta * np.array([1, x2]) * t2)

In [ ]:
rv = multivariate_normal(m, S_N)

plt.imshow(rv.pdf(pos), extent=[-1, 1, -1 ,1], cmap='inferno')
plt.scatter(a[0], a[1], marker='+', c='white', s=100)

plt.title('Posterior distribution')

plt.xlabel('w_0')
plt.ylabel('w_1')

In [ ]:
posterior_samples = rv.rvs(6)

In [ ]:
plt.figure(figsize=[4,4])

xs = np.linspace(-1, 1, 50)

for s in posterior_samples:
    plt.plot(xs, s[0] + xs * s[1])

plt.scatter([x, x2], [t, t2], marker='o')    

plt.xlim([-1,1])
plt.ylim([-1,1])

Of course we're not actually interested in the value of w itself but rather in making predictions of t for new values of x. Therefore we need to evaluate the *predictive* distribution.

Turns out the predictive distribution is also a gaussian because of result 2.115 in Bishop (convolution of two Gaussian distributions). 

In [ ]:
# evaluate predictive distribution. 



means = []
variances = [] 
for x in xs:

    phi = np.array([1, x])

    predictive_mean = np.dot(m, phi)
    predictive_var = 1 / beta + np.dot(phi, np.matmul(S_N, phi))

    #predictive_dist = multivariate_normal(predictive_mean, predictive_var)
    
    means.append(predictive_mean)
    variances.append(predictive_var)
    
    
means = np.array(means)
variances = np.array(variances)

In [ ]:
plt.figure(figsize=[4,4])

plt.plot(xs, means)
plt.fill_between(xs, means - np.sqrt(variances), means + np.sqrt(variances), alpha=.2)
plt.plot(xs, get_y(xs, a))

plt.xlim([-1,1])
plt.ylim([-1,1])